# About
This notebook contains the workflow to obtain details from an image to be stored as a JSON document in the MongoDB Storage.

Productionze:
* Prompt functions
* LLM API

# Libraries

In [1]:
import os
import json
from pathlib import Path
import time


import pandas as pd
from datetime import datetime

import google.generativeai as genai

from PIL import Image

# User-Defined Functions

## ■ API functions

In [2]:
def google_api_test(googleai_api_key):
    
    genai.configure(api_key=googleai_api_key)
    
    genai_model = genai.GenerativeModel(
        model_name='models/gemini-1.5-pro-latest',
    )

    try:
        response = genai_model.generate_content("Return: 'Ok'")
        return response.candidates[0].content.parts[0].text.replace(" \n", "")
        
    except Exception as e:
        return e

## ■ Job ID

In [3]:
def get_current_datetime():
    now = datetime.now()
    formatted_now = now.strftime("%y%m%d%H%M-%s")
    return formatted_now

def get_first_letters(name):
    # Split the name into words
    words = name.split()
    # Get the first letter of each word
    first_letters = [word[0] for word in words if word]
    # Join the first letters to form a string
    result = ''.join(first_letters)
    return result

def create_job_id(user_name):
    return get_current_datetime() + '-'+ get_first_letters(user_name)
    

## ■ GoogleAI responses

### > Text-to-text

In [4]:
def t2t(text):
    response = genai_model.generate_content(text)
    return response.text

### > Image-to-text

In [5]:
def i2t(img):
    """
    image to text
    """
    response = genai_model.generate_content(img)
    
    return response.text

### > Text & Image-to-text

In [6]:
def ti2t(text, img):
    """
    The generate_content method can handle a wide variety of use cases depending on what the underlying model supports, including:
    * multi-turn chat
    * multimodal input. 
    
    The available models only support text and images as input, and text as output.
    """
    response = genai_model.generate_content([text, img])
    
    return response.text

### > Dummy JSON Parser

In [7]:
def dummy_json_parser(dirty_json):
    # Dummy function for text to JSON parser
    parser = dirty_json
    parser = parser.replace("\n", "")
    parser = parser.replace("json{", "{")
    parser = parser.replace("```", "")
    return json.loads(parser)

## ■ Image Processing

In [8]:
def concatenate_images_in_folder_vertically(folder_path, output_path):
    # Get all image files in the folder, excluding the output file
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'gif', 'bmp')) and f != os.path.basename(output_path)]
    
    # Load all images
    images = [Image.open(os.path.join(folder_path, img)) for img in image_files]
    
    # Determine the dimensions of the final image
    max_width = max(image.width for image in images)
    total_height = sum(image.height for image in images)
    
    # Create a new image with the calculated dimensions
    combined_image = Image.new('RGB', (max_width, total_height))
    
    # Paste all images into the new image
    current_y = 0
    for image in images:
        combined_image.paste(image, (0, current_y))
        current_y += image.height
    
    # Save the combined image
    combined_image.save(output_path)

## ■ Retrying Failed AI Functions

In [9]:

def retry_function(func, retries=3, delay=45):
    """
    Retries a function if it fails.
    
    Parameters:
    - func: the function to retry
    - retries: the number of retry attempts
    - delay: the delay in seconds between retries
    
    Returns:
    - The result of the function if it succeeds
    - None if all retries fail
    """
    for attempt in range(retries):
        try:
            result = func()
            return result
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:  # If it's not the last attempt
                print(f"Waiting {delay} seconds before retrying...")
                time.sleep(delay)
    print("All attempts failed.")
    return None


## ■ Prompts

In [61]:
def label_text_extraction():
    image_path = 'images/labels/combined_image.png'
    
    clothing_picture = {
        'mime_type': 'image/png',
        'data': Path(image_path).read_bytes()
    }

    prompt = """ Read through the text in the image, and structure your response as a JSON object with the following keys. Return only English text and don't break lines within your responses:
    
    * brand: Analyze the provided image of a clothing label. Identify and extract the brand name from the label. Must be composed only of alphanumeric characters. The brand name is usually a prominent and distinctive word or logo, often displayed in a larger font than other text, but not necessarily. It may include a unique font or design. If unclear, return an empty value. 
    * materials: The types and percentages of fibers used in the fabric. Make a key-value pairing with the type of material use for this item and its percentage as value. If unclear, return an empty value. 
    * care_instructions: Analyze the provided image of a clothing label and extract detailed care instructions for the garment, particularly numerical temperature information in Celsius. Include guidelines on how to wash, dry, iron, and maintain the garment to ensure its longevity and quality. Look for specific temperature recommendations for washing and ironing if available. Return a structured response with key-value pairs where the keys represent the care instruction categories ('washing', 'bleaching', 'drying', 'ironing', 'dry_clean') and the values contain the corresponding detailed instructions. If information is unclear, return an empty value.
    * size: The size of the garment which can be indicated by letters (e.g., S, M, L, XL), or a nationality abbreviation (e.g., UK, US, EU, MX) followed by a corresponding numerical size (e.g., UK 8, US 10, EU 36). Return a list with all relevant information, ensuring that it strictly follows the previously specified format. Trade any dot (or any other character) for a space to adjust relevant information to the previous formatting (e.g. 'M', 'UK 8'). If unclear, return an empty value. 
    * country_origin: In what country the garment was made. Usually preceded by a text 'Made in ...'. If not explicit, return an empty value.
    """
    
    response = genai_model.generate_content([prompt, clothing_picture])
    
    return dummy_json_parser(response.text)

In [126]:
def cloth_description(label_json):
    """
    To do: Create a grid of every perspective of the clothing item to analyze. 
    So far, this is for a single image. 
    """
    
    image_path = 'images/clothes/dirty_shirt.png'
    
    clothing_picture = {
        'mime_type': 'image/png',
        'data': Path(image_path).read_bytes()
    }

    prompt = f""" Structure the image description as a JSON object with the following keys:
    
    * clothing_type: Return a single value categorizing the clothing type. 
    * usage_suggestions: Analyze the provided image of a clothing label and determine the top 5 most common usages for the clothing item. Ignore any damage, focus on the clothing item. Consider the style, material, and any other relevant details to suggest appropriate contexts in which this item might be worn. Possible usages might include `casual`, `weddings`, `business formal`, `sports`, `outdoor activities`, `beachwear`, `night out`, etc. Return a list of the most relevant and appropriate usage contexts for this item. If unclear, return an empty value.
    * audience: State the most likely fashion audience. Choose between 'Men', 'Women', 'Kids', or 'Gender Neutral'.
    * colors: Return a dictionary with a 'primary_color', a list of 'secondary_colors', and a list of where the locations for the secondary colors. If there are no secondary colors just return an empty field.
    * practical_description: Using the information from this JSON object (provided within []), and additional data (requested as a Python dictionary command for the label_json variable), redact a sentence with great syntax and congruency. Specifically, follow this structure: ```[Clothing Item] size {label_json['size'][0]} for ['audience'] by {label_json['brand']} is primarily [primary_color] with [secondary_colors] accents on [Secondary colors locations]. Made of {', '.join(f'{value} {key}' for key, value in label_json['materials'].items())}```. Ensure correct syntax and formatting without double spaces or non-alphanumeric characters.
    * sales_pitch: Analyze the clothing labels and describe the clothing item in a compelling sales pitch. Highlight key features, benefits, and unique selling points without using phrases like `our item` or `our product`. Consider aspects such as material, design, comfort, versatility, and any special features or certifications. Appeal to potential buyers by emphasizing why this item would be a valuable addition to their wardrobe. Mention potential usages, quality, and standout characteristics. If information is unclear, create a general appealing description based on typical clothing attributes.
    * damage: Make a list with one or multiple visible damages relative to if the item were new. Choose between 'Rips, Tears, Holes', 'Stained', 'Fading', 'Missing Buttons', 'Loose Threads' one or more. If there is no noticeable damage, just return an empty field.
    * explanation: Redact a short explanation structured as a nested dictionary for the reason behind each of the previous labelings. One sentence for each[clothing_type] 
    """
    
    response = genai_model.generate_content([prompt, clothing_picture])
    
    return dummy_json_parser(response.text)

# Test

In [83]:
# TEST
googleai_api_key = pd.read_csv(r'/home/cesar/Coding/GoogleAI-API-key.txt', header=None)[0][0]
google_api_test(googleai_api_key)
genai_model = genai.GenerativeModel(model_name='models/gemini-1.5-pro-latest')

In [13]:
user_name = "César Alejandro Pozo Luyo"

In [15]:
folder_path = 'images/labels/'
output_path = folder_path + 'combined_image.png'
concatenate_images_in_folder_vertically(folder_path, output_path)

In [62]:
label_json = label_text_extraction()

In [127]:
# Note: An edit must be done to process an image grid
clothes_json = cloth_description(label_json)

In [128]:
job_id = create_job_id(user_name)
main_json = {"Job_ID": job_id}
main_json.update(label_json) 
main_json.update(clothes_json) 
main_json

{'Job_ID': '2405291626-1717021582-CAPL',
 'brand': 'Sfera',
 'materials': {'cotton': '100%'},
 'care_instructions': {'washing': 'Wash maximum 30°C - Mild process',
  'bleaching': 'Do not bleach',
  'drying': 'Do not tumble dry',
  'ironing': 'Iron at maximum sole plate temperature of 110°C - Iron inside out',
  'dry_clean': 'Do not dry clean'},
 'size': ['M', 'MEX 38'],
 'country_origin': 'India',
 'clothing_type': 'T-Shirt',
 'usage_suggestions': ['casual'],
 'audience': 'Kids',
 'colors': {'primary_color': 'white',
  'secondary_colors': ['red'],
  'secondary_colors_location': ['neckline', 'sleeves']},
 'practical_description': 'T-Shirt size M for Kids by Sfera is primarily white with red accents on neckline and sleeves. Made of 100% cotton.',
 'sales_pitch': 'This classic white T-shirt is made from 100% breathable cotton, ensuring comfort and durability. Its timeless design features a playful contrast red trim on the neckline and sleeves, adding a touch of style to a versatile wardro